In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error, r2_score
import pandas_profiling as pp
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import xgboost
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
pd.set_option('display.max_columns', 1000)

In [4]:
def onehotencode(data,col_name,prefix):
    return pd.concat([data, pd.get_dummies(data[col_name], prefix=prefix)],axis=1)

In [5]:
def encode_total_stops(data):
    data['Encoded_stops'] = 0.0
    for i in range(0,len(data['Total_Stops'])):
        if data.loc[i, 'Total_Stops'] == '1 stop':
            data.loc[i,'Encoded_stops'] = 0.1
        elif data.loc[i, 'Total_Stops'] == '2 stops':
            data.loc[i,'Encoded_stops'] = 0.2
        elif data.loc[i, 'Total_Stops'] == '3 stops':
            data.loc[i,'Encoded_stops'] = 0.3
        elif data.loc[i, 'Total_Stops'] == '4 stops':
            data.loc[i,'Encoded_stops'] = 0.4
        elif data.loc[i, 'Total_Stops'] == 'non-stop':
            data.loc[i,'Encoded_stops'] = 0.0

    return data

In [6]:
def extract_journey_data(data):
    data['Journey_Day'] = pd.DataFrame(pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.day)
    data['Journey_Month'] = pd.DataFrame(pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.month)
    data['Journey_day_of_week'] =  pd.DataFrame(((pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.dayofweek)).astype('int'))
    data['Journey_over_weekend'] =  pd.DataFrame(((pd.to_datetime(data['Date_of_Journey'], format='%d/%m/%Y').dt.dayofweek) // 5).astype('int'))

    
    return data

In [100]:
data['Date_of_Journey'].head()

0    24/03/2019
1     1/05/2019
2     9/06/2019
3    12/05/2019
4    01/03/2019
Name: Date_of_Journey, dtype: object

In [7]:
def encode_duration(data):
    data['Duration_Type'] = 0
    for i in range(0, len(data)):
    
        split_len = data.loc[i,'Duration'].split(' ')
        if len(split_len) == 1:
            if str(data.loc[i,'Duration']).find('h') != -1:
#                print ("hours: ",data.loc[i,'Duration'])
                hours = int(split_len[0].replace('h',""))
                mins = 0
            else:
                hours = 4 
#                print ("mins: ",data.loc[i,'Duration']
                mins = int(split_len[0].replace('m',""))
        else:
            
            hours = int(split_len[0].replace('h',""))
            mins  = int(split_len[1].replace('m',""))
        
        data.loc[i,'Duration_Type'] = (hours*60 + mins)
        
    return data

In [8]:
def flight_hours(data):
    data['Fly_Hours'] = 0
    
    data_Dep_Time = data['Dep_Time'].str.split(":",n=1,expand=True).astype('int64')
    data_Dep_Time.columns = ['Hours','Mins']
    for i in range(0,len(data)):
        
        hr = data_Dep_Time.loc[i, 'Hours']
        min = data_Dep_Time.loc[i, 'Mins']
        data.loc[i,'Fly_Hours'] = (hr*60 + (min))
            
    return data

In [9]:
def arrival_hours(data):
    data['Arrival_Hours'] = 0
    for i in range(0,len(data)):
      
        data_Arr_Time = data.loc[i,'Arrival_Time'].split(" ")[0]
        data_Dep_Time = data_Arr_Time.split(":")
        
        hr = int(data_Dep_Time[0])
        min = int(data_Dep_Time[1])
        data.loc[i,'Arrival_Hours'] = (hr*60 + (min))
            
    return data

In [10]:
def log_transform(data):
    return np.log10(data)

# Data cleaning and Modeling starts here

In [11]:
data = pd.read_excel('C:\\Users\\LENOVO\\Desktop\\Flight_Ticket_Participant_Datasets\\Data_Train.xlsx')

In [12]:
test = pd.read_excel('C:\\Users\\LENOVO\\Desktop\\Flight_Ticket_Participant_Datasets\\Test_set.xlsx')

In [13]:
data_copy = data
test_copy = test

In [14]:
data_copy.shape

(10683, 11)

In [15]:
data_copy.drop_duplicates(keep='first', inplace=True)

In [16]:
data_copy.shape

(10463, 11)

In [17]:
#Drop anamolies.
#data_copy = data_copy[data_copy['Price'] > data["Price"].quantile(0.01)]
#data_copy = data_copy[data_copy['Price'] < data["Price"].quantile(0.99)]
data_copy = data_copy[data_copy['Airline'] != 'Trujet']
data_copy = data_copy[data_copy['Additional_Info'] != 'Red-eye flight']

In [18]:
#data_copy.loc[data_copy['Airline'] == 'Jet Airways Business','Airline'] = 'Jet Airways'
#data_copy.loc[data_copy['Airline'] == 'Multiple carriers Premium economy','Airline'] = 'Multiple carriers'
#data_copy.loc[data_copy['Airline'] == 'Vistara Premium economy','Airline'] = 'Vistara'

In [ ]:
data_copy = extract_journey_data(data_copy)

In [109]:
data_copy['Month_Start'] = 0
data_copy.loc[data_copy['Journey_Day'] == 1,'Month_Start'] = 1
data_copy.loc[data_copy['Journey_Day'] == 3,'Month_Start'] = 1
data_copy.loc[data_copy['Journey_Day'] == 6,'Month_Start'] = 1
data_copy.loc[data_copy['Journey_Day'] == 9,'Month_Start'] = 1

KeyError: 'Month_Start'

In [19]:
data_copy.loc[data_copy['Destination'] == 'New Delhi','Destination'] = 'Delhi'

In [20]:
data_copy.reset_index(inplace=True)

In [21]:
data_copy = data_copy[data_copy['Airline'] != 'Trujet']

In [22]:
data_copy = onehotencode(data_copy,'Airline',"Airline_")

In [23]:
data_copy.columns

Index(['index', 'Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price', 'Airline__Air Asia', 'Airline__Air India',
       'Airline__GoAir', 'Airline__IndiGo', 'Airline__Jet Airways',
       'Airline__Multiple carriers', 'Airline__SpiceJet', 'Airline__Vistara'],
      dtype='object')

In [24]:
data_copy = onehotencode(data_copy,'Source',"Source_")

In [25]:
data_copy = onehotencode(data_copy,'Destination',"Destination_")

In [26]:
#handle No Info, No info values before one hot encoding.
data_copy.loc[data_copy['Additional_Info'] == 'No info','Additional_Info'] = 'No Info'
data_copy.loc[data_copy['Additional_Info'] == '1 Long layover','Additional_Info'] = 'Others'
data_copy.loc[data_copy['Additional_Info'] == '1 Short layover','Additional_Info'] = 'Others'
data_copy.loc[data_copy['Additional_Info'] == '2 Long layover','Additional_Info'] = 'Others'
data_copy.loc[data_copy['Additional_Info'] == 'Business class','Additional_Info'] = 'Others'
data_copy.loc[data_copy['Additional_Info'] == 'Change airports','Additional_Info'] = 'Others'


data_copy = onehotencode(data_copy,'Additional_Info',"Additional_Info_")

In [27]:
data_copy = encode_total_stops(data_copy)

In [29]:
data_copy = encode_duration(data_copy)

In [30]:
data_copy = flight_hours(data_copy)

In [31]:
data_copy = arrival_hours(data_copy)

In [32]:
#data_copy = onehotencode(data_copy,'Fly_Hours',"Fly_Hours_")

In [33]:
data_final = data_copy

In [34]:
data_final.loc[data_final['Encoded_stops'].isnull(),'Encoded_stops'] = 3

In [35]:
data_final.drop(['Airline','Date_of_Journey','Source','Destination','Route','Dep_Time','Arrival_Time','Duration',
                'Total_Stops','Additional_Info'],axis=1,inplace=True)
#                'Total_Stops','Additional_Info','Fly_Hours'],axis=1,inplace=True)

In [36]:
data_final.drop(['index'],axis=1,inplace=True)

In [37]:
train_Feature = data_copy[[x for x in data_final.columns if (x != 'Price' and x != 'index')]]
train_Target = data_copy['Price']

In [38]:
train_Feature.columns

Index(['Airline__Air Asia', 'Airline__Air India', 'Airline__GoAir',
       'Airline__IndiGo', 'Airline__Jet Airways', 'Airline__Multiple carriers',
       'Airline__SpiceJet', 'Airline__Vistara', 'Source__Banglore',
       'Source__Chennai', 'Source__Delhi', 'Source__Kolkata', 'Source__Mumbai',
       'Destination__Banglore', 'Destination__Cochin', 'Destination__Delhi',
       'Destination__Hyderabad', 'Destination__Kolkata',
       'Additional_Info__In-flight meal not included',
       'Additional_Info__No Info',
       'Additional_Info__No check-in baggage included',
       'Additional_Info__Others', 'Encoded_stops', 'Journey_Month',
       'Journey_day_of_week', 'Journey_over_weekend', 'Duration_Type',
       'Fly_Hours', 'Arrival_Hours'],
      dtype='object')

In [39]:
X_train, X_test, y_train, y_test = train_test_split(train_Feature, train_Target, test_size=0.30, random_state=101)

In [40]:
null_columns=X_train.columns[X_train.isnull().any()]
X_train[null_columns].isnull().sum()

Series([], dtype: float64)

In [41]:
X_train.head()

,Airline__Air Asia,Airline__Air India,Airline__GoAir,Airline__IndiGo,Airline__Jet Airways,Airline__Multiple carriers,Airline__SpiceJet,Airline__Vistara,Source__Banglore,Source__Chennai,Source__Delhi,Source__Kolkata,Source__Mumbai,Destination__Banglore,Destination__Cochin,Destination__Delhi,Destination__Hyderabad,Destination__Kolkata,Additional_Info__In-flight meal not included,Additional_Info__No Info,Additional_Info__No check-in baggage included,Additional_Info__Others,Encoded_stops,Journey_Month,Journey_day_of_week,Journey_over_weekend,Duration_Type,Fly_Hours,Arrival_Hours
6949,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0.0,6,4,0,90,835,925
8604,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0.0,6,6,1,170,1435,165
883,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0.1,5,3,0,690,450,1140
4116,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0.1,3,4,0,920,1320,800
5606,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0.1,6,3,0,540,600,1140


In [42]:
scaler = StandardScaler()

In [43]:
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=train_Feature.columns)

In [44]:
X_test =  pd.DataFrame(scaler.transform(X_test),columns=train_Feature.columns)

# Final Prediction on Test set starts here

In [45]:
#test_copy.loc[test_copy['Airline'] == 'Jet Airways Business','Airline'] = 'Jet Airways'
#test_copy.loc[test_copy['Airline'] == 'Multiple carriers Premium economy','Airline'] = 'Multiple carriers'
#test_copy.loc[test_copy['Airline'] == 'Vistara Premium economy','Airline'] = 'Vistara'

In [46]:
test_copy.loc[test_copy['Destination'] == 'New Delhi','Destination'] = 'Delhi'

In [47]:
test_copy = onehotencode(test_copy,'Airline',"Airline_")

In [48]:
test_copy = onehotencode(test_copy,'Source',"Source_")

In [49]:
test_copy = onehotencode(test_copy,'Destination',"Destination_")

In [50]:
#handle No Info, No info values before one hot encoding.
test_copy.loc[test_copy['Additional_Info'] == 'No info','Additional_Info'] = 'No Info'
test_copy.loc[test_copy['Additional_Info'] == '1 Long layover','Additional_Info'] = 'Others'
test_copy.loc[test_copy['Additional_Info'] == '1 Short layover','Additional_Info'] = 'Others'
test_copy.loc[test_copy['Additional_Info'] == '1 Long layover','Additional_Info'] = 'Others'
test_copy.loc[test_copy['Additional_Info'] == 'Business class','Additional_Info'] = 'Others'
test_copy.loc[test_copy['Additional_Info'] == 'Change airports','Additional_Info'] = 'Others'

test_copy = onehotencode(test_copy,'Additional_Info',"Additional_Info_")

In [ ]:
test_copy = extract_journey_data(test_copy)

In [ ]:
test_copy['Month_Start'] = 0
test_copy.loc[test_copy['Journey_Day'] == 1,'Month_Start'] = 1
test_copy.loc[test_copy['Journey_Day'] == 3,'Month_Start'] = 1
test_copy.loc[test_copy['Journey_Day'] == 6,'Month_Start'] = 1
test_copy.loc[test_copy['Journey_Day'] == 9,'Month_Start'] = 1

In [ ]:
test_copy['is_event'] = 0
test_copy.loc[[test_copy['Month'] == 3 and test_copy['Day'] == 3],'is_event'] = 1
test_copy.loc[[test_copy['Month'] == 4 and test_copy['Day'] == 21],'is_event'] = 1
test_copy.loc[[test_copy['Month'] == 5 and test_copy['Day'] == 18],'is_event'] = 1

In [51]:
test_copy = encode_total_stops(test_copy)

In [53]:
test_copy = encode_duration(test_copy)

In [54]:
test_copy = flight_hours(test_copy)

In [55]:
test_copy = arrival_hours(test_copy)

In [56]:
#test_copy = onehotencode(test_copy,'Fly_Hours',"Fly_Hours_")

In [57]:
test_final = test_copy

In [58]:
data_copy.head()

,Price,Airline__Air Asia,Airline__Air India,Airline__GoAir,Airline__IndiGo,Airline__Jet Airways,Airline__Multiple carriers,Airline__SpiceJet,Airline__Vistara,Source__Banglore,Source__Chennai,Source__Delhi,Source__Kolkata,Source__Mumbai,Destination__Banglore,Destination__Cochin,Destination__Delhi,Destination__Hyderabad,Destination__Kolkata,Additional_Info__In-flight meal not included,Additional_Info__No Info,Additional_Info__No check-in baggage included,Additional_Info__Others,Encoded_stops,Journey_Month,Journey_day_of_week,Journey_over_weekend,Duration_Type,Fly_Hours,Arrival_Hours
0,3897,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0.0,3,6,1,170,1340,70
1,7662,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0.2,5,2,0,445,350,795
2,13882,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0.2,6,6,1,1140,565,265
3,6218,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0.1,5,6,1,325,1085,1410
4,13302,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0.1,3,4,0,285,1010,1295


In [59]:
data_copy['Encoded_stops'].unique()

array([ 0. ,  0.2,  0.1,  0.3,  0.4])

In [60]:
test_final.drop(['Airline','Date_of_Journey','Source','Destination','Route','Dep_Time','Arrival_Time','Duration',
                'Total_Stops','Additional_Info'],axis=1,inplace=True)
#                'Total_Stops','Additional_Info','Fly_Hours'],axis=1,inplace=True)

In [61]:
test_final =  pd.DataFrame(scaler.transform(test_final),columns=test_final.columns)

In [62]:
test_final.columns

Index(['Airline__Air Asia', 'Airline__Air India', 'Airline__GoAir',
       'Airline__IndiGo', 'Airline__Jet Airways', 'Airline__Multiple carriers',
       'Airline__SpiceJet', 'Airline__Vistara', 'Source__Banglore',
       'Source__Chennai', 'Source__Delhi', 'Source__Kolkata', 'Source__Mumbai',
       'Destination__Banglore', 'Destination__Cochin', 'Destination__Delhi',
       'Destination__Hyderabad', 'Destination__Kolkata',
       'Additional_Info__In-flight meal not included',
       'Additional_Info__No Info',
       'Additional_Info__No check-in baggage included',
       'Additional_Info__Others', 'Encoded_stops', 'Journey_Month',
       'Journey_day_of_week', 'Journey_over_weekend', 'Duration_Type',
       'Fly_Hours', 'Arrival_Hours'],
      dtype='object')

In [63]:
train_Feature.columns

Index(['Airline__Air Asia', 'Airline__Air India', 'Airline__GoAir',
       'Airline__IndiGo', 'Airline__Jet Airways', 'Airline__Multiple carriers',
       'Airline__SpiceJet', 'Airline__Vistara', 'Source__Banglore',
       'Source__Chennai', 'Source__Delhi', 'Source__Kolkata', 'Source__Mumbai',
       'Destination__Banglore', 'Destination__Cochin', 'Destination__Delhi',
       'Destination__Hyderabad', 'Destination__Kolkata',
       'Additional_Info__In-flight meal not included',
       'Additional_Info__No Info',
       'Additional_Info__No check-in baggage included',
       'Additional_Info__Others', 'Encoded_stops', 'Journey_Month',
       'Journey_day_of_week', 'Journey_over_weekend', 'Duration_Type',
       'Fly_Hours', 'Arrival_Hours'],
      dtype='object')

In [64]:
test_final.shape

(2671, 29)

In [65]:
train_Feature.shape

(10461, 29)

In [66]:
y_train_log =  y_train.apply(log_transform)

In [67]:
y_test_log =  y_test.apply(log_transform)